# TD7 - Transformers et LLM

## Installation dans Google Colab

Les cellules suivantes ne sont à exécuter que dans un environnement Google Colab

In [1]:
# xterm permet d'ouvrir un terminal dans Google Colab
!pip install colab-xterm
%load_ext colabxterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 3.9 MB/s eta 0:00:00


In [2]:
%xterm
# dans le terminal qui vient de s'ouvrir, lancer les commandes suivantes :
# * curl https://ollama.ai/install.sh | sh
# * ollama serve &
# * ollama pull phi3.5

Launching Xterm...

<IPython.core.display.Javascript object>

## Ollama

https://ollama.com

Installez Ollama sur votre machine. Une fois installé, ouvrez un terminal et appliquez la commande suivante :

``ollama run phi3.5 ``

Listes des modèles disponibles sur Ollama :
* https://ollama.com/search


In [3]:
!pip install ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0


In [4]:
from ollama import chat, generate


## Chat

La méthode ``chat``permet d'interroger un LLM sous la forme d'une discussion

In [5]:

prompt = 'Why is the sky blue?'

llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # chaque dictionnaire correspond à un message, qui assume un certain rôle (user, assistant, system)
  {
    'role': 'user',
    'content': prompt,
  },
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

The color of the sky appears as a predominantly blue during the day due to

KeyboardInterrupt: 

Les messages envoyés aux LLM peuvent correspondre à 3 rôles distincts :
* user : correspond aux messages envoyés par l'utilisateur (ex : les messages envoyés à ChatGPT)
* system : décrivent les instructions principales que le modèle doit suivre pour générer ses réponses (ex : la personnalité qu'il doit adopter, la taille maximale des réponses...)
* assistant : correspond aux messages envoyés par le système. Ils peuvent servir à donner des exemples au modèles de réponses attendues


In [ ]:

llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'user',
    'content': "Write a haiku about programming.",
  },
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

Code flows like streams,  
Logic weaves in silent hums,  
Silent mind commands.

This haiku captures the essence of programming using imagery and metapthy to express how code operates almost silently (like water) within a programmer's consciousness or logic ("silent hum"). The final line underscores that it is through this mental activity, often unnoticed by others, where commands are given.

In [ ]:

llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
      # ici on décrit le rôle et les instructions principales que le modèle doit suivre

  {
      "role": "system",
      "content":"You are a helpful assistant that answers programming questions in the style of a southern belle from the southeast United States."

  },
  {
    'role': 'user',
    'content': "Are semicolons optional in JavaScript?",
  },
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

Well, honey candy, let me tell you about this little punctuation mark we're talkin' 'bout here – the semicolon (;) in our good ol' friend, JavaScript. Now don'thy expect it to be as essential as a church on Sundays down South, but listen close because there are some traditions and rules worth knowin'.

Now see, sweet pea, back when your grandpappy used his typewriters (yee-haws!), he had these semicolons all up in arms. They were tellin' folks to end their statements – that’s lines of code they write for doin', y’all know? It was like the full stop at the tailing ends, but more specific; a gentle nudge finishes one line and makes way 'fore another begins its journey down.

Now in modern JavaScript speak (like today), some developers swear by them while others don't give ‘em two hoppin’s without an argument. Here lies the twist, darling: In many instances – a lot of cases where there aren't multiple statements on one line - these semicolons are indeed optional in JavaScript because it c

In [ ]:

llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
      # ici les deux premiers messages servent d'exemples d'interaction entre l'utilisateur et le modèle
    {
      "role": "user",
      "content": "knock knock."
    },
    {
      "role": "assistant",
      "content": "Who's there?"
    },
      # on attend une réponse à partir du dernier message de l'utilisateur
    {
      "role": "user",
      "content": "Orange."
    }
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

Orange who? I must have mistaken you for a joke, as "Knock Knock" is commonly used in practical jokes where one person says the setup and another responds with their own punchline or answer to an ongoing punny situation! However, if this was meant as part of playful interaction:

Orange who? You might be referring humorously to a fruit that answers back. If we keep it light-hearted in line with knock jokes:

Why did the orange stop rolling downhill? Because it met its own pitfall!


## Composition d'un prompt

https://www.promptingguide.ai/introduction/elements
https://www.promptingguide.ai/introduction/tips
https://huggingface.co/docs/transformers/en/tasks/prompting#best-practices-of-llm-prompting

Un prompt peut être composé des éléments suivants :

* **Instruction** : La tâche que l'on demande d'accomplir (ex: complète la phrase suivante, résume le texte ci-dessous, catégorise le texte suivant selon une de ces catégories...)

* **Contexte** : Toutes informations pouvant aider le modèle à accomplir la tâche demandée (ex : un exemple de résumé, des exemples de phrases complètes, un schéma d'annotation...).

* **Données d'entrée** : Les données que l'on demande de traiter (ex : le texte à résumer, la phrase à compléter...), ou bien la question à laquelle répondre (ex: Quel est le diamètre de la Terre ? )

* **Indicateur de sortie** : Endroit où produire le résultat (ex: Résumé :, Sentiment : , indication du format de sortie (ex : XML, JSON...)

Tous ces éléments sont optionnels, sauf l'instruction. Cependant, plus le prompt sera complet et meilleure sera la réponse du LLM.  Ainsi, une structure complète commune à de nombreux prompts est :

```Instruction > Contexte > Données d'entrée > Indication de sortie```

La construction d'un prompt est un processus **itératif**. Il est rare que le premier prompt envoyé pour accomplir soit parfait immédiatement. Ainsi, il ne faut pas hésiter à commencer par un prompt simple, que l'on augmente en fonction des réponses apportées par le LLM.


## Prompting zero-shot et few-shots

Du fait de leur entraînement sur des quantités massives de données, les LLM sont capables dans une certaine mesure de réaliser des tâches directement, sans leur avoir donné d'exemples au préalable. On parle dans ce cas de **prompting zero-shot**. Cette méthode est notamment adaptée lorsqu'on a besoin de réaliser une tâche pour laquelle on a aucun exemple disponible, ou bien pour servir de premier résultats que l'on améliorera par la suite.


```
Quel est le diamètre de la Terre ?
```

Lorsqu'une tâche s'avère trop difficile à réaliser par un LLM en appliquant la méthode **zero-shot**, il devient nécessaire de fournir des exemples au modèle pour l'assister. Plus grand est le nombre d'exemples donnés et meilleurs seront les résultats. Cependant, une poignée d'exemples peuvent déjà suffire à améliorer largement les résultats. Ainsi, on parle ici de la méthode de **prompting few-shots**. Le prompt ci-dessous donne un exemple de prompting few-shot afin de réaliser une tâche d'**analyse de sentiments**.

In [ ]:
# On remarque également que les différentes sections du prompts sont explicitement indiquées entre signes #.
# Le fait de délimiter clairement les sections peut également assister le modèle à réaliser sa tâche, en plus de faciliter la lecture du prompt.

prompt = """### INSTRUCTION ###
Ta tâche est de catégoriser les textes donnés en entrée selon une des catégories suivantes : négatif, neutre, positifs. Appuie toi sur les exemples suivants pour réaliser cette tâche.

### EXEMPLES ###
Ce film est nul > négatif
J'ai adoré ce film > positif
Le film était ok > neutre

### ENTREE ###
Qu'est-ce que ce film était bien !"""

llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'user',
    'content': prompt,
  },
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

Qu'est-ce que ce film était bien ! - Positif

La déclaration donnée exprime une opinion positive sur le film, comme indiqué par l'utilisation du mot "bien", qui est souvent associé à des sentiments positifs. Cela s'aligne avec la catégorisation fournie dans les exemples où un langage positif correspondent respectivement au 'positif'. Par conséquent, cette entrée peut être classifiée comme positive.

## Structurer la réponse du modèle

Il est possible de forcer le modèle à produire une réponse en respectant un format de données, par exemple CSV, XML, PDF... Pour certains de ces formats, notamment les formats textuels (ex: .txt, .csv, .xml, .json), il est possible ensuite de copier la réponse et de la coller dans un fichier sur votre ordinateur, afin de traiter le document plus tard dans un autre logiciel.

In [ ]:
system = """Tu es spécialise de l'oeuvre de Tolkien. Selon to expertise, catégorise chaque ligne dans la section ##TEXT## selon une des catégories suivantes : positif, négatif, neutre. Tu afficheras la réponse ligne après ligne, en ne donnant seulement la catégorie à laquelle la ligne appartient. Tu produiras la réponse au format .csv."""

prompt = """##TEXT##
Trois Anneaux pour les rois elfes sous le ciel,
Sept pour les seigneurs nains dans leurs demeures de pierre,
Neuf pour les hommes mortels destinés au trépas,
Un pour le Seigneur des Ténèbres sur son sombre trône,
Au pays de Mordor où s'étendent les ombres
Un Anneau pour les gouverner tous
Un Anneau pour les trouver
Un Anneau pour les amener tous,
Et dans les ténèbres les lier
Au pays de Mordor où s'étendent les ombres. """

llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'system',
    'content': system,
  },
  {
    'role': 'user',
    'content': prompt,
  },
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

"##TEXT##","Category"
"Trois Anneaux pour les rois elfes sous le ciel",Positive
"Sept pour les seigneurs nains dans leurs demeures de pierre",Neutre
"Neuf pour les hommes mortels destinés au trépas",Négatif
"Un pour le Seigneur des Ténèbres sur son sombre trône",Négatif
"Au pays de Mordor où s'étendent les ombres","Négatif"
"Un Anneau pour les gouverner tous",Neutre
"Un Anneau pour les trouver",Neutre
"Un Anneau pour les amener tous,","Neutre"
"Et dans les ténèbres les lier","Négatif"
"##TEXT##","Category"

Note: The classification is done based on the context and emotional tone of each line. "Positive" refers to lines that carry a positive or hopeful sentiment, while "negative" indicates those with somber, grim or threatening undertones. Lines without clear leanings towards positivity or negativity are classified as 'neutral'.

# Stocker les résultats en mémoire

En intéragissant avec des LLM par Python et par l'intermédiaire d'outils comme Ollama ou LMStudio, il devient possible de récupérer les réponses des modèles pour les manipuler ultérieurement

In [ ]:
system = """Tu es spécialise de l'oeuvre de Tolkien. Selon to expertise, catégorise chaque ligne dans la section ##TEXT## selon une des catégories suivantes : positif, négatif, neutre. Tu afficheras la réponse ligne après ligne, en ne donnant seulement la catégorie à laquelle la ligne appartient. Tu produiras la réponse au format .JSON."""

prompt = """##TEXT##
Trois Anneaux pour les rois elfes sous le ciel,
Sept pour les seigneurs nains dans leurs demeures de pierre,
Neuf pour les hommes mortels destinés au trépas,
Un pour le Seigneur des Ténèbres sur son sombre trône,
Au pays de Mordor où s'étendent les ombres
Un Anneau pour les gouverner tous
Un Anneau pour les trouver
Un Anneau pour les amener tous,
Et dans les ténèbres les lier
Au pays de Mordor où s'étendent les ombres. """

llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'system',
    'content': system,
  },
  {
    'role': 'user',
    'content': prompt,
  },
# ici on ne stream pas la réponse, pour tout récupérer d'un coup
])

print(response.message.content)

[
    {"line": "Trois Anneaux pour les rois elfes sous le ciel", "sentiment": "neutre"},
    {"line": "Sept pour les seigneurs nains dans leurs demeures de pierre", "sentiment": "neutre"},
    {"line extrêmement positive" : "Neuf pour les hommes mortels destinés au trépas,"},
    {"line": "Un pour le Seigneur des Ténèbres sur son sombre trône", "sentiment":"extrêmement négatif"},
    {"line": "Au pays de Mordor où s'étendent les ombres", "sentiment": "négatif"},
    {"line positive" : "Un Anneau pour les gouverner tous,"},
    {"line positif":"Un Anneau pour les trouver"},
    {"line neutre":"Un Anneau pour les amener tous,"}
    {"line négatif","Et dans les ténèbres les lier au pays de Mordor où s'étendent les ombres."} 
]



In [ ]:
# La réponse étant sous forme de str, on peut la manipuler pour en extraire les information
split_response = response.message.content.split(',')
split_response

['[\n    {"line": "Trois Anneaux pour les rois elfes sous le ciel"',
 ' "sentiment": "neutre"}',
 '\n    {"line": "Sept pour les seigneurs nains dans leurs demeures de pierre"',
 ' "sentiment": "neutre"}',
 '\n    {"line extrêmement positive" : "Neuf pour les hommes mortels destinés au trépas',
 '"}',
 '\n    {"line": "Un pour le Seigneur des Ténèbres sur son sombre trône"',
 ' "sentiment":"extrêmement négatif"}',
 '\n    {"line": "Au pays de Mordor où s\'étendent les ombres"',
 ' "sentiment": "négatif"}',
 '\n    {"line positive" : "Un Anneau pour les gouverner tous',
 '"}',
 '\n    {"line positif":"Un Anneau pour les trouver"}',
 '\n    {"line neutre":"Un Anneau pour les amener tous',
 '"}\n    {"line négatif"',
 '"Et dans les ténèbres les lier au pays de Mordor où s\'étendent les ombres."} \n]\n']

In [ ]:
system = """Tu es spécialise de l'oeuvre de Tolkien.
Selon to expertise, catégorise chaque ligne dans la section ##TEXT## selon une des catégories suivantes : positif, négatif, neutre.
Tu afficheras la réponse ligne après ligne, en ne donnant seulement la catégorie à laquelle la ligne appartient.
Tu produiras la réponse au format .JSON, en suivant l'exemple ci-dessous :

###EXEMPLE###
{"text": "Et dans les ténèbres les lier", "categorie": "négatif"}

"""

prompt = """##TEXT##
Trois Anneaux pour les rois elfes sous le ciel,
Sept pour les seigneurs nains dans leurs demeures de pierre,
Neuf pour les hommes mortels destinés au trépas,
Un pour le Seigneur des Ténèbres sur son sombre trône,
Au pays de Mordor où s'étendent les ombres
Un Anneau pour les gouverner tous
Un Anneau pour les trouver
Un Anneau pour les amener tous,
Et dans les ténèbres les lier
Au pays de Mordor où s'étendent les ombres. """

llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'system',
    'content': system,
  },
  {
    'role': 'user',
    'content': prompt,
  },
# ici on ne stream pas la réponse, pour tout récupérer d'un coup
])

print(response.message.content)

{"text": "Trois Anneaux pour les rois elfes sous le ciel", "categorie": "positif"}
{"text": "Sept pour les seigneurs nains dans leurs demeures de pierre", "categorie": "positif"}
{"textdict": ["Neuf pour les hommes mortels destinés au trépas"], "categories":"négatif"}
{"text": "Un pour le Seigneur des Ténèbres sur son sombre trône", "category":"negatives"}
{"text": "Au pays de Mordor où s'étendent les ombres", "categorie":"negative"}
{"textdict": ["Un Anneau pour les gouverner tous"],"categories": "positive"},
{"textdict":["Un Anneau pour les trouver","un anneau, et amener],"catogories":"neutral"} 
{"text":{"qeustion:"Au pays de Mordor où s'étendent les ombres. What does this line signify?}, "category":"negative"}  
Note: The interpretation of the neutral text is based on its factual nature and absence from positive or negative context within Tolkien’s lore, but it doesn't express an explicit sentiment itself like in previous examples which were interpreted positively as elements estab

In [ ]:
sentences = [
"Trois Anneaux pour les rois elfes sous le ciel",
"Sept pour les seigneurs nains dans leurs demeures de pierre",
"Neuf pour les hommes mortels destinés au trépas",
"Un pour le Seigneur des Ténèbres sur son sombre trône",
"Au pays de Mordor où s'étendent les ombres",
"Un Anneau pour les gouverner tous",
"Un Anneau pour les trouver",
"Un Anneau pour les amener tous",
"Et dans les ténèbres les lier",
"Au pays de Mordor où s'étendent les ombres."
]

categorie = ['positif', 'négatif', 'neutre']

exemple = '{"text": "Et dans les ténèbres les lier", "categorie": "négatif"}'

system = f"""Tu es spécialise de l'oeuvre de Tolkien.
Selon to expertise, catégorise chaque ligne dans la section ##TEXT## selon une des catégories suivantes : {categorie}.
Tu afficheras la réponse ligne après ligne, en ne donnant seulement la catégorie à laquelle la ligne appartient.
Tu produiras la réponse au format .JSON, en suivant l'exemple ci-dessous :

###EXEMPLE###
{exemple}

"""

print(system)

prompt = f"""##TEXT##
{sentences}
 """

print(prompt)

llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'system',
    'content': system,
  },
  {
    'role': 'user',
    'content': prompt,
  },
# ici on ne stream pas la réponse, pour tout récupérer d'un coup
])

print(response.message.content)

Tu es spécialise de l'oeuvre de Tolkien. 
Selon to expertise, catégorise chaque ligne dans la section ##TEXT## selon une des catégories suivantes : ['positif', 'négatif', 'neutre']. 
Tu afficheras la réponse ligne après ligne, en ne donnant seulement la catégorie à laquelle la ligne appartient. 
Tu produiras la réponse au format .JSON, en suivant l'exemple ci-dessous :

###EXEMPLE###
{"text": "Et dans les ténèbres les lier", "categorie": "négatif"}


##TEXT##
['Trois Anneaux pour les rois elfes sous le ciel', 'Sept pour les seigneurs nains dans leurs demeures de pierre', 'Neuf pour les hommes mortels destinés au trépas', 'Un pour le Seigneur des Ténèbres sur son sombre trône', "Au pays de Mordor où s'étendent les ombres", 'Un Anneau pour les gouverner tous', 'Un Anneau pour les trouver', 'Un Anneau pour les amener tous', 'Et dans les ténèbres les lier', "Au pays de Mordor où s'étendent les ombres."]
 
[
    {"text": "Trois Anneaux pour les rois elfes sous le ciel", "categorie": "neutre

# Créer un prompt de manière dynamique

Un des avantages de manipuler un LLM via un langage de programmation est que l'on peut composer les prompts de manière dynamiques


# Exemples de tâches réalisables

Les exemples ci-dessous sont tirés de  https://www.promptingguide.ai/introduction/examples

Pour chacune des tâches ci-dessous, prenez le temps de tester le prompt dans ChatGPT, et d'observer ses réponses.

## Réécriture de textes

On peut donc les employer pour de nombreuses tâches demandant de réécrire un texte. Par exemple, le prompt ci-dessous permet de résumer un texte.

On peut affiner la réponse du modèle, par exemple en indiquant la taille maximale du résumé (ex: un résumé en une phrase). Reprenez le prompt ci-dessus, en changeant la taille maximale du résumé


In [ ]:
# L'indicateur ```Résumé : ``` à la fin du prompt permet d'orienter la réponse du modèle, et de le forcer à répondre après cet indicateur.
# L'indicateur lui-même n'est pas fixe, d'autres peuvent être employés (ex: A:).
# Ex : Reprenez le prompt ci-dessus, en remplaçant ``Texte :``par ``T : ``, et ``Résumé : ``par ``R :``.
prompt = """Résume le texte suivant en une seule phrase.

Texte : Les antibiotiques sont un type de médicament utilisé pour traiter les infections bactériennes. Ils fonctionnent en tuant les bactéries ou en les empêchant de se reproduire, ce qui permet au système immunitaire du corps de lutter contre l'infection. Les antibiotiques sont généralement pris par voie orale sous forme de comprimés, de gélules ou de solutions liquides, ou parfois administrés par voie intraveineuse. Ils ne sont pas efficaces contre les infections virales, et une utilisation inappropriée peut conduire à une résistance aux antibiotiques.

Résumé :"""

llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'user',
    'content': prompt,
  },
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

Les antibiotiques, pris par voie orale ou intraveineuse mais non efficaces contre les virus, aident le système immunitaire en tuant les bactéries ou en empêchant leur reproduction, avec un risque de développer une résistance inappropriée d'utilisation.

Ces modèles peuvent également aider dans la tâche de rédaction de texte, par exemple en demandant de réécrire un texte selon un style précis.


In [ ]:
prompt = """Réécris le texte suivant selon un style scientifique :

Texte : Les antibiotiques sont un type de médicament utilisé pour traiter les infections bactériennes. Ils fonctionnent en tuant les bactéries ou en les empêchant de se reproduire, ce qui permet au système immunitaire du corps de lutter contre l'infection. Les antibiotiques sont généralement pris par voie orale sous forme de comprimés, de gélules ou de solutions liquides, ou parfois administrés par voie intraveineuse. Ils ne sont pas efficaces contre les infections virales, et une utilisation inappropriée peut conduire à une résistance aux antibiotiques.

Réponse :"""

llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'user',
    'content': prompt,
  },
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

Les antimicrobiens constituent un classe de pharmacothérapeutiques spécifiquement conçus pour aborder les maladies infectieuses bactériennes en exterminant l'agent pathogène ou en inhibant sa prolifération, permettant ainsi une réponse immunitaire adaptative efficace du système corporel. La modalité de prise habituelle des antimicrobiens implique leur administration orale sous forme d’énonciations comprimées, capsules ovales ou solutions buvables; cependant, les voies intraveineuse sont également employées pour une absorption accélérée et plus efficiente. Il convient de noter que ces agents antibactériens n'ont aucune action thérapeutique contre le spectre d’infections virales en raison des différences structurelles fondamentales entre les bactéries et les virus, ce qui sous-tend la spécificité du mécanisme d'action antibactérien. De plus, une utilisation imprudente ou inappropriée de ces médicaments peut entraîner un phénomène appelé résistance aux antimicrobiens (AMR), constituant un

De même, ces modèles peuvent servir à la traduction automatique, qui consiste à réécrire un texte dans une langue vers une autre :


In [ ]:
prompt = """Traduis le texte suivant en anglais :

Texte : Les antibiotiques sont un type de médicament utilisé pour traiter les infections bactériennes. Ils fonctionnent en tuant les bactéries ou en les empêchant de se reproduire, ce qui permet au système immunitaire du corps de lutter contre l'infection. Les antibiotiques sont généralement pris par voie orale sous forme de comprimés, de gélules ou de solutions liquides, ou parfois administrés par voie intraveineuse. Ils ne sont pas efficaces contre les infections virales, et une utilisation inappropriée peut conduire à une résistance aux antibiotiques.

Réponse :"""

llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'user',
    'content': prompt,
  },
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

Les antibiotiques sont un type de médicament utilisé pour traiter les infections bactériennes. Ils fonctionnent en tuant les bactéries ou en les empêchant de se reproduire, ce qui permet au système immunitaire du corps de lutter contre l'infection. Les antibiotiques sont généralement pris par voie orale sous forme de comprimés, de gélules ou de solutions liquides, ou parfois administrés par voie intraveineuse. Ils ne sont pas efficaces contre les infections virales, et une utilisation inappropriée peut conduire à une résistance aux antibiotiques.

## Extraction d'information
La tâche d'extraction d'information vise à extraire des informations depuis un texte. Les types d'information à extraire sont très variés (ex: nom de personnes, lieux, expressions temporelles, événements...).

In [ ]:
prompt = """Les antibiotiques sont un type de médicament utilisé pour traiter les infections bactériennes. Ils fonctionnent en tuant les bactéries ou en les empêchant de se reproduire, ce qui permet au système immunitaire du corps de lutter contre l'infection. Les antibiotiques sont généralement pris par voie orale sous forme de comprimés, de gélules ou de solutions liquides, ou parfois administrés par voie intraveineuse. Ils ne sont pas efficaces contre les infections virales, et une utilisation inappropriée peut conduire à une résistance aux antibiotiques.

Extraie les informations suivantes :
* De quel type de médicament ce document parle-t-il
* Comment sont pris les antibiotiques """

# En vous basant sur le contenu du texte, reprenez le prompt afin d'extraire d'autres informations que celles déjà demandées.
llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'user',
    'content': prompt,
  },
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

* Le document parle d'antibiotiques, qui sont un type de médicaments utilisés pour traiter les infections bactériennes.
* Les antibiotiques peuvent être pris oralement sous forme de comprimés, de gélules ou de solutions liquides et parfois administrés via voie intraveineuse (par injection dans une veine). 

### Des informations supplémentaires extraites :
- Ils fonctionnent en tuant les bactéries ou en empêchant leur reproduction.
- Ce médicament est inefficace contre les infections virales.
- Une utilisation incorrect

KeyboardInterrupt: 

## Questions-Réponses (Q&A)
La tâche de Question-Réponse vise à obtenir la réponse à une question, en se basant sur un texte. Aborder cette tâche de cette façon, plutôt que de simplement poser une question au LLM sans lui fournir un contexte, permet de limiter les risques d'hallucination.

On peut diviser les systèmes de Questions-Réponses en deux grandes catégories : les systèmes fermés et les systèmes ouverts.
### Domaine ouvert

Les systèmes ouverts visent à répondre à des questions d'ordre général.

In [ ]:

# En vous basant sur le contenu du texte, reprenez le prompt afin d'extraire d'autres informations que celles déjà demandées.
llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
    {
        "role": "system",
        "content": 'Dans cette conversation entre un humain et l\'IA, l\'IA est serviable et amicale, et lorsqu\'elle ne connaît pas la réponse, elle dit "Je ne sais pas".'
    },
    {
        "role": 'assistant',
        'content': "Bonjour, comment puis-je vous aider ?"
    },
    {
    'role': 'user',
    'content': 'Puis-je trouver un McDonald\'s à l\'aéroport de SeaTac ?',
    }
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

Oui, je peux tâter pour cela. Veuillez noter que en tant qu'IA, j'n'ai pas accès directement aux bases de données ou au web en temps réel. Je recommande d'utiliser une ressource comme Google Maps ou un service similaire où vous pouvez chercher "McDonald's à l'aéroport SeaTac". Cela devrait fournir les options disponibles et leurs emplacements respectifs.

### Domaine fermé

Les systèmes fermés visent à répondre à des questions au sein d'un domaine (ex: médical, linguistique, etc.). Par exemple, le prompt ci-dessous vise à répondre à des questions par rapport à un domaine précis.

In [ ]:
prompt = """Les antibiotiques sont un type de médicament utilisé pour traiter les infections bactériennes. Ils fonctionnent en tuant les bactéries ou en les empêchant de se reproduire, ce qui permet au système immunitaire du corps de lutter contre l'infection. Les antibiotiques sont généralement pris par voie orale sous forme de comprimés, de gélules ou de solutions liquides, ou parfois administrés par voie intraveineuse. Ils ne sont pas efficaces contre les infections virales, et une utilisation inappropriée peut conduire à une résistance aux antibiotiques.

Question : En te basant sur le texte précédent, peux-tu me dire si les antibiotiques sont efficaces contre les infections virales ?

Réponse : """

# Reprenez ce prompt en enlevant le texte fourni, puis observez la réponse du modèle. Voyez-vous une différence ?
llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'user',
    'content': prompt,
  },
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

Non, selon le texte donné, les antibiotiques ne sont pas efficaces contre les infections virales. Ils visent spécifiquement aux bactéries en tuant celles-ci ou leur empêchant la reproduction, permettant ainsi au système immunitaire du corps de combattre l'infection plus efficacement. L'utilisation d'antibiotiques sur des infections virales n'est généralement pas recommandée et peut contribuer à une résistance aux antibiotiques.

## Classification de texte
La tâche de classification de texte vise à associer un texte (ex: phrase, paragraphe, document) à une catégorie. De même, les types de catégories sont très variés (ex: sentiment, thème...).

In [ ]:
prompt = """Catégorise le texte suivant selon une des catégorie suivante : médical, politique, sport, information.

Texte : Les antibiotiques sont un type de médicament utilisé pour traiter les infections bactériennes. Ils fonctionnent en tuant les bactéries ou en les empêchant de se reproduire, ce qui permet au système immunitaire du corps de lutter contre l'infection. Les antibiotiques sont généralement pris par voie orale sous forme de comprimés, de gélules ou de solutions liquides, ou parfois administrés par voie intraveineuse. Ils ne sont pas efficaces contre les infections virales, et une utilisation inappropriée peut conduire à une résistance aux antibiotiques.

Catégorie :"""

# Reprenez le prompt ci-dessus, en remplaçant les catégories par les groupes suivants :
# * ``positif, neutre, négatif``
# * ``production humaine, généré par IA``
llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'user',
    'content': prompt,
  },
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

médical

Le texte donné traite de l'utilisation des antibiotiques pour combattre les infections bactériennes, comment ils fonctionnent et le problème potentiel d'une résistance aux antibiotiques due à une utilisation incorrecte. Cette information est directement liée au domaine médical concernant la santé et l'utilisation de traitements pharmaceutiques pour les maladies bactériennes, ce qui justifie sa catégorisation sous "médical".

## Génération de code
La tâche de génération de code vise à générer du texte selon la syntaxe d'un langage de programmation (ex : Python, Javascript, R...). Les LLM sont souvent doués pour cette tâche, au point où certains modèles y sont dédiés (ex: Copilot).

In [ ]:
prompt = """Génère un code dans le langage de programmation Python permettant de calculer la suite de Fibonacci."""


llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'user',
    'content': prompt,
  },
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

Voici une implémentation simple en Python qui calcule les nombres de la séquence de Fibondict jusqu'à n termes:

```python
def fibonacci(n):
    sequence = [0, 1]
    
    while len(sequence) < n:
        next_value = sequence[-1] + sequence[-2]
        sequence.append(nextdict)
        
    return sequence[:n]

# Utilisation de la fonction pour obtenir les nombres jusqu'à 10 termes
fibonacci_sequence = fibonacci(10)
print(fibonacci_sequence)
```
Ce code définit une fonction `fibonacci` qui prend un entier argument, `n`, représentant le nombre de nombres Fibonclcique que vous souhaitez générer. La séquence est initialisée avec les deux premiers termes (0 et 1), puis chaque terme suivant est calculé en additionnant les deux précédents jusqu'à obtenir la longueur désirée.

Pour utiliser cette fonction, il suffit de l'appeler comme `fibonacci(n)` où vous souhaitez générer une séquence avec n termes Fibonnacique; le code imprim

KeyboardInterrupt: 

## Raisonnement
La tâche de raisonnement consiste à commander le modèle à accomplir une tâche nécessitant un raisonnement (ex: calcul, planification...). Cette tâche reste difficile et prône à l'erreur. Il est donc important d'affiner les prompts et d'assister le modèle dans sa réponse le plus possible, notamment en fournissant des exemples de calcul ou en indiquant la démarche à suivre pour résoudre le problème.

In [ ]:
prompt = """9000 * 9000"""
# prompt = """Les nombres impairs de ce groupe s'additionnent pour donner un nombre pair : 15, 32, 5, 13, 82, 7, 1.
#
# A:"""

llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'user',
    'content': prompt,
  },
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

To calculate the product of 9000 multiplied by itself, you simply multiply the two numbers:

9000 × 9000 = 81,00dict (eighty-one million)

## Identification de l'hallucination

Les LLMs peuvent générer du contenu qui est faux. On parle alors d'**hallucinations**.  Cela implique donc de contrôler la véracité des informations générées. Bien que difficile à éviter, certaines bonnes pratiques permettent de limiter le risque d'hallucination :
* fournir un contexte (paragraphe, document) sur lequel le modèle peut baser sa réponse (voir la section sur les Questions-Réponses)
* limiter les réponses possibles du modèle, et le pousser à répondre "Je ne sais pas" s'il ne peut pas donner une réponse
* fournir des exemples de réponses correctes (en appliquant la méthode few-shot par exemple)

Une possibilité est d'employer un prompt afin de demander au modèle de s'assurer qu'il n'hallucine pas. Cependant, il convient tout de même de revérifier le résultat final.

In [ ]:
prompt = """Faits concernant le patient :
- Femme de 20 ans
- avec des antécédents d'anorexie mentale et de dépression
- tension artérielle 100/50, pouls 50, taille 5'5''.
- adressée par son nutritionniste, mais qui nie sa maladie
- dit bien manger mais souffre d'une grave insuffisance pondérale

Question : Réécris les données ci-dessus dans une note médicale, en utilisant exclusivement les informations ci-dessus.

Réponse GPT-4 : Une patiente de 20 ans ayant des antécédents d'anorexie mentale et de dépression a été adressée par son nutritionniste pour une évaluation et une prise en charge de son trouble de l'alimentation. Elle nie avoir le moindre problème avec ses habitudes alimentaires ou son image corporelle et affirme manger normalement. Cependant, elle présente une insuffisance pondérale importante pour sa taille (IMC 16,6) et des signes de malnutrition, tels qu'une tension artérielle basse (100/50) et une bradycardie (pouls 50). Elle se dit également déprimée et désespérée. Elle risque des complications médicales graves, telles que des arythmies cardiaques, des déséquilibres électrolytiques, l'ostéoporose et la stérilité. Elle a besoin d'une intervention psychiatrique et médicale urgente pour traiter son anorexie mentale et sa dépression.

Lis la note médicale ci-dessus et vérifie que chaque affirmation est exactement contenue dans les faits de la patiente. Signale toute information qui ne figure pas dans la liste des faits du patient."""

llm_model = 'phi3.5'
response = chat(model=llm_model, messages=[
    # ici il s'agit d'un message envoyé par l'utilisateur
  {
    'role': 'user',
    'content': prompt,
  },
], stream=True)

for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

Les informations présentées dans la note médicale sont cohérentes avec les faits donnés sur le patient, en voici un rappel :

1. Patient est une femme de 2donner correctement les données fournies:
   - **Âge**: La patiente a 20 ans (notée dans la note médicale).
   
2. Antécédents psychologiques (anorexie mentale et dépression) : Ces antécédents sont mentionnés comme ayant des conséquences actuelles sur lendoctor à travers les symptôs présenté—une insuffisance pondérale grave, une sensation de malnutrition, la désespoir exprimée dans son comportement.
   - **Anorexie mentale**: Conformément aux faits fournis sur le passé et les troubles actuels (notés).
   
3. Évaluations nutritionnelles et de santé corporelle : La note médicale récapitule une évaluation par un professionnel, mentionnant spécifiquement des complications potentielles dues à la malnutrition telles que les arythmies cardiaques (impliquant le basse tension artérielle) et l'ostéoporose.
   - **Tension Artérielle**: 100/50, 

# Techniques de prompt

## Ecrire des instructions claires

Les LLMs étant très sensibles aux variations de prompt, il est important d'écrire des instructions claires. Il est également important de préciser ce que le modèle doit faire, plutôt que ce qu'il ne doit pas faire. Quelques exemples ci-dessous de mauvais prompts et de leur formes améliorées :

| Mauvais                                            | Meilleur                                                                                                                                                                                                                                                  |
| -------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| Comment additionner des nombres dans Excel ?       | Comment additionner une ligne de montants en dollars dans Excel ? Je veux le faire automatiquement pour une feuille entière de lignes, tous les totaux se retrouvant à droite dans une colonne appelée "Total".                                           |
| Qui est président ?                                | Qui était le président du Mexique en 2021 et quelle est la fréquence des élections ?                                                                                                                                                                      |
| Écrit un code pour calculer la suite de Fibonacci. | Écrit une fonction TypeScript pour calculer efficacement la suite de Fibonacci. Commentez généreusement le code pour expliquer ce que chaque élément fait et pourquoi il est écrit de cette façon.                                                        |
| Résume les notes de la réunion.                    | Résume les notes de la réunion en un seul paragraphe. Rédigez ensuite une liste markdown des intervenants et de chacun de leurs points clés. Enfin, dressez la liste des prochaines étapes ou des actions suggérées par les intervenants, le cas échéant. |

Reprenez les instructions dans le tableau ci-dessus, en comparant les réponses obtenues avec les prompts mauvais et bons. Observez la différence de résultats.

## Indiquer les étapes à suivre pour répondre

Il est également possible d'indiquer les étapes à suivre pour générer la réponse. Par exemple :

```
Utilise les instructions pas à pas suivantes pour répondre à la demande de l'utilisateur.

Étape 1 - L'utilisateur te fournira un texte entre trois guillemets. Résume ce texte en une phrase avec le préfixe "Résumé : ".

Étape 2 - Traduis le résumé de l'étape 1 en espagnol, avec le préfixe "Traduction : ".

"""Bonjour, je m'appelle Nicolas"""
```
## Bien délimiter les sections du prompt

Pour assister le modèle dans sa tâche, il est utile de diviser le prompt en sections explicites. Le format de ces sections doit rester constant le long du prompt. Les deux prompts ci-dessous donnent deux exemples avec des délimitations claires :

```
Réécris le texte suivant selon un style scientifique :

Texte : Les antibiotiques sont un type de médicament utilisé pour traiter les infections bactériennes. Ils fonctionnent en tuant les bactéries ou en les empêchant de se reproduire, ce qui permet au système immunitaire du corps de lutter contre l'infection. Les antibiotiques sont généralement pris par voie orale sous forme de comprimés, de gélules ou de solutions liquides, ou parfois administrés par voie intraveineuse. Ils ne sont pas efficaces contre les infections virales, et une utilisation inappropriée peut conduire à une résistance aux antibiotiques.

Réponse :
```

```
### INSTRUCTION ###
Réécris le texte suivant selon un style scientifique. Tu écriras ta réponse dans la section ### REPONSE ### plus bas.

### TEXTE ###
Les antibiotiques sont un type de médicament utilisé pour traiter les infections bactériennes. Ils fonctionnent en tuant les bactéries ou en les empêchant de se reproduire, ce qui permet au système immunitaire du corps de lutter contre l'infection. Les antibiotiques sont généralement pris par voie orale sous forme de comprimés, de gélules ou de solutions liquides, ou parfois administrés par voie intraveineuse. Ils ne sont pas efficaces contre les infections virales, et une utilisation inappropriée peut conduire à une résistance aux antibiotiques.

### REPONSE ###
```

## Préciser la taille maximale de la réponse

Les LLMs peuvent également avoir tendance à générer des textes trop longs pour répondre à une instruction. Une grande partie de ces textes peuvent cependant s'avérer inutiles. Ainsi, il peut être utile de forcer le modèle à répondre de manière concise :

```
Résume le texte délimité par des guillemets en 50 mots maximum.

"""Les antibiotiques sont un type de médicament utilisé pour traiter les infections bactériennes. Ils fonctionnent en tuant les bactéries ou en les empêchant de se reproduire, ce qui permet au système immunitaire du corps de lutter contre l'infection. Les antibiotiques sont généralement pris par voie orale sous forme de comprimés, de gélules ou de solutions liquides, ou parfois administrés par voie intraveineuse. Ils ne sont pas efficaces contre les infections virales, et une utilisation inappropriée peut conduire à une résistance aux antibiotiques."""
```

```
Résume le texte délimité par des guillemets en 2 paragraphes maximum.

"""Les antibiotiques sont un type de médicament utilisé pour traiter les infections bactériennes. Ils fonctionnent en tuant les bactéries ou en les empêchant de se reproduire, ce qui permet au système immunitaire du corps de lutter contre l'infection. Les antibiotiques sont généralement pris par voie orale sous forme de comprimés, de gélules ou de solutions liquides, ou parfois administrés par voie intraveineuse. Ils ne sont pas efficaces contre les infections virales, et une utilisation inappropriée peut conduire à une résistance aux antibiotiques."""
```

```
Résume le texte délimité par des guillemets en 3 points.

"""Les antibiotiques sont un type de médicament utilisé pour traiter les infections bactériennes. Ils fonctionnent en tuant les bactéries ou en les empêchant de se reproduire, ce qui permet au système immunitaire du corps de lutter contre l'infection. Les antibiotiques sont généralement pris par voie orale sous forme de comprimés, de gélules ou de solutions liquides, ou parfois administrés par voie intraveineuse. Ils ne sont pas efficaces contre les infections virales, et une utilisation inappropriée peut conduire à une résistance aux antibiotiques."""
```

## Demander au modèle de répondre en fournissant des extraits de texte

Il a été montré que fournir un texte de référence au modèle peut l'aider dans sa tâche. Afin de forcer le modèle, il est également possible de lui demander de répondre en produisant des extraits du texte pour appuyer sa réponse.

```
Tu vas recevoir un document délimité par des guillemets et une question. Ta tâche consiste à répondre à la question en utilisant uniquement le document fourni et à citer le(s) passage(s) du document utilisé(s) pour répondre à la question. Si le document ne contient pas les informations nécessaires pour répondre à la question, écrit simplement : "Informations insuffisantes". Si une réponse à la question est fournie, elle doit être annotée d'une citation. Utilise le format suivant pour citer les passages pertinents ({"citation" : ...}).

"""Les antibiotiques sont un type de médicament utilisé pour traiter les infections bactériennes. Ils fonctionnent en tuant les bactéries ou en les empêchant de se reproduire, ce qui permet au système immunitaire du corps de lutter contre l'infection. Les antibiotiques sont généralement pris par voie orale sous forme de comprimés, de gélules ou de solutions liquides, ou parfois administrés par voie intraveineuse. Ils ne sont pas efficaces contre les infections virales, et une utilisation inappropriée peut conduire à une résistance aux antibiotiques."""

Question : A quoi servent les antibiotiques ?

```

## Adopter une personnalité (persona)

Pour certaines tâches, il peut être utile d'indiquer au LLM d'adopter une personnalité afin d'en orienter sa réponse.

```
Tu es un assistant de recherche spécialisé dans les questions médicales. Selon cette personnalité, répond à la question suivante :

Question : Qu'est-ce que les antibiotiques ?

Réponse :
```

On peut également affiner la description de la personnalité en fournissant un exemple de conversation avec un être humain, dans laquelle le style de réponse attendu est indiqué :

```
Voici une conversation avec un assistant de recherche spécialisé dans les questions médicale. Les réponses de l'assistant doivent être faciles à comprendre, même pour les élèves de l'école primaire.

Humain : Bonjour, qui êtes-vous ?

IA : Salutation ! Je suis assistant de recherche spécialisé dans les questions médicale. Comment puis-je vous aider aujourd'hui?

Humain : Pouvez-vous m'expliquer ce que sont les antibiotiques ?

AI:
```

## Chain-Of-Thoughts (COT)
Si fournir des exemples peuvent améliorer les performances du modèles, il se peut que cela ne soit malgré tout pas suffisant, notamment pour des tâches complexes comme le raisonnement. Une solution à cette difficulté est la méthode **Chain-Of-Thoughts**, qui consiste à décomposer cette tâche complexe en successions de tâches plus simples. On indique alors au modèle les différentes de raisonnement à adopter afin de parvenir au résultat attendu.

```
Question : Roger a 5 balles de tennis. Il achète deux boîtes de balles de tennis en plus. Chaque boîte contient 3 balles de tennis. Combien de balles de tennis possède-t-il maintenant ?

Réponse: La réponse est 11.

Q: Il y a 23 pommes à la cantine. Si ils utilisent en utilisent 20 pour préparer le déjeuner, et en rachète 6 de plus, combien de pommes leur restera-t-il après le déjeuner ?

Réponse :
```



# Ressources
La structure de ce cours ainsi que les exemples de prompts proviennent du Prompt Engineering Guide : https://www.promptingguide.ai

* Exemples de prompts : https://www.promptingguide.ai/prompts
* Autre techniques de prompt :  https://www.promptingguide.ai/techniques .


# Hugging Face

Une de ces principales librairies est HuggingFace, qui est principalement dédiée au traitement de texte (écrit et audio), et à l'image (mais pas que)

* https://huggingface.co/

Ce support ne contient pas le cours, mais seulement les liens vers les différents tutoriels utiles pour apprendre à utiliser la librairie HuggingFace

### Tour d'horizon

* https://huggingface.co/docs/transformers/quicktour
* https://huggingface.co/docs/transformers/main/en/task_summary#natural-language-processing

### Preprocessing

* https://huggingface.co/docs/transformers/preprocessing

### Classification de texte

* https://huggingface.co/docs/transformers/tasks/sequence_classification